# Optical Character Recognition (OCR) Topps Sticker

## Libraries and settings

In [25]:
# Libraries
import os
import re
import cv2
import pytesseract
from pytesseract import Output
import PIL
from PIL import Image
import fnmatch
import tempfile
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/Users/ivesbrunner/Documents/Studium/01_Bachelor/04_Semester/04_ScientificProgramming/euro_2024_trends/sticker_collector


## Read data from image

### Open and crop image

In [26]:
# Open the image
image_path = './stickers/sticker_example.jpeg'
image = Image.open(image_path)

# Crop the image to show only the right half
width, height = image.size
cropped_image = image.crop((1100, 800, width, height-500))

### Read team and number sticker

In [27]:
text = pytesseract.image_to_string(cropped_image, config='--psm 11')

# replace all multiple linebreaks to one
text = re.sub(r'\n+', '\n', text)
team = text.split('\n')[0]
number = text.split('\n')[1]

print("Found sticker with the following information:")
print("Team: ", team)
print("Number: ", number)

Found sticker with the following information:
Team:  ESP
Number:  20


## Add sticker to collection if not owned yet (Postgres)

### Create database connection

In [28]:
import psycopg2

# Database connection details
host = 'localhost'
port = '5432'
database = 'postgres'
user = 'admin'
password = 'secret'

# Establish a connection
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

### Fetch stickers

In [29]:
# Create a cursor object
cursor = conn.cursor()

# Execute SQL queries
cursor.execute("SELECT * FROM stickers ORDER BY team ASC, number ASC;")
result = cursor.fetchall()

# Print the result
print("Current stickers in the database:")

if (len(result) == 0):
    print("No stickers found in the database.")

for row in result:
    print(row)

# Close the cursor and connection
cursor.close()

Current stickers in the database:
(20, 'ESP')


### Store sticker to database

In [30]:
cursor = conn.cursor()

# Check if the sticker exists in the database
cursor.execute("SELECT * FROM stickers WHERE team = %s AND number = %s", (team, number))
result = cursor.fetchall()

# If the sticker does not exist, insert it into the database
if not result:
    cursor.execute("INSERT INTO stickers (team, number) VALUES (%s, %s)", (team, number))
    conn.commit()
    print("Sticker added to the database.")
else:
    print("Sticker already exists in the database.")

cursor.close()

Sticker already exists in the database.


### Close database connection

In [31]:

conn.close()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [32]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.5.0
Datetime: 2024-05-27 14:38:54
Python Version: 3.10.13
-----------------------------------
